### Training of modified U-Net for Optic Disc on DRISHTI-GS database (cross-validation fold #0).

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

from model import *
from process_images import *
from process_result import *

print('Keras version:', keras.__version__)
print('TensorFlow version:', tf.__version__)

#### DRISHTI-GS

Accessing data, preparing train/validation sets division:

In [ ]:
h5f = h5py.File(os.path.join(os.path.dirname(os.getcwd()), 'data', 'hdf5_datasets', 'DRISHTI_GS.hdf5'), 'r')
suf = '_20'
#-----------------------------------------------------------------------------------------------------------#
img_test = h5f['DRISHTI-GS-test/512 px/img_cropped' + suf]
cup_test = h5f['DRISHTI-GS-test/512 px/cup_cropped' + suf]
disc_test = h5f['DRISHTI-GS-test/512 px/disc_cropped' + suf]

img_train = h5f['DRISHTI-GS/512 px/img_cropped' + suf]
cup_train = h5f['DRISHTI-GS/512 px/cup_cropped' + suf]
disc_train = h5f['DRISHTI-GS/512 px/disc_cropped' + suf]

### Train images

In [ ]:
train_images, train_cups, train_discs = create_all_color_list(img_train, cup_train, disc_train)

In [ ]:
img_size = IMG_SIZE
epochs = NUM_EPOCHS
spe = SPE
arq_name = 'drishti_all_colors'
prefix_results = 'all_colors'
folder_name = root_path(arq_name)

arch_name_cup = "Cup, log_dice loss"
arch_name_disc = "Disc, log_dice loss"

In [ ]:
X_test, Y_test = next(data_generator(img_test, cup_test, img_size, train_or_test='test', batch_size=100, stationary=True))

weights_folder = os.path.join(folder_name, 'models_weights', '{},{}'.format(datetime.now().strftime('%d.%m,%H-%M'), arch_name_cup))

model = create_compile_model(img_size, 7e-4)
history_cup = train_cup(train_images, 
                    train_cups, 
                    'cup_'+ prefix_results, 
                    model, epochs, X_test, Y_test, img_size, spe, weights_folder)

In [ ]:
pred_iou, pred_dice, pred_cup = predict(img_test, X_test, Y_test, model, img_size)
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, prefix_results + '_mean_iou_dice_cup.csv')
save_predict_images(folder_name, 'cups', test_idx, pred_cup)

In [ ]:
X_test, Z_test = next(data_generator(img_test, disc_test, img_size, train_or_test='test', batch_size=100, stationary=True))
model_disc = create_compile_model(img_size, 1e-3)
weights_folder = os.path.join(folder_name, 'models_weights', '{},{}'.format(datetime.now().strftime('%d.%m,%H-%M'), arch_name_disc))

history_disc = train_disc(train_images, 
                     train_discs, 
                     'disc_'+ prefix_results, 
                     model_disc, epochs, X_test, Z_test, img_size, spe, weights_folder)

In [ ]:
pred_iou, pred_dice, pred_disc = predict(img_test, X_test, Z_test, model_disc, img_size)
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, prefix_results + '_mean_iou_dice_disc.csv')
save_predict_images(folder_name, 'discs', test_idx, pred_disc)

In [ ]:
plot_results(history_cup, epochs, folder_name, arq_name + '_cup')

In [ ]:
plot_results(history_disc, epochs, folder_name, arq_name + '_disc')

In [ ]:
df, diametros = create_table_result(pred_cup, pred_disc, test_idx, folder_name, prefix_results)

In [ ]:
print(disc)

In [ ]:
print(diametros)